<b> Definizione di tutti gli import necessari e dei path utilizzati durante le varie fasi </b>

In [1]:
import librosa as lb
import librosa.display as display
import librosa.effects as effects
import numpy as np
import csv as csv
import os
import matplotlib.pyplot as plt
import pandas as pd
from IPython.display import Audio

from hmmlearn import hmm
from collections import Counter

import sys
n_cluster = 1000


BASE_TRAINING_PATH="C:\\Users\\alexp\\Desktop\\Voxceleb_wav\\wav\\"
#BASE_TRAINING_PATH="/home/axel1143/Scrivania/Voxceleb_wav/wav/"
BASE_PATH = "C:\\Users\\alexp\\Desktop\\Voxceleb_mat\\"
#BASE_PATH = "/home/alex/Scrivania/Voxceleb_mat/"

<b> Definizione della funzione di pre-processing in due passi: </b>

1) Fase di pre-enfasi con a = 0.97
2) Fase di riduzione dei momenti di silenzio con threshold = 25db

In [2]:

def pre_processing(path):
  #load
  audio, sr = lb.load(path = path, mono = True)

  #pre-emp
  pre_emp_audio = effects.preemphasis(audio, 0.97)


  #removing silence in audio
  clips = effects.split(pre_emp_audio, top_db=25)
  silenced=[]
  for c in clips:
    data = audio[c[0]: c[1]]
    silenced.extend(data)

  silence = np.array(silenced)



  return silence, sr



<b> Creazione del dataset: </b>

1) Applicazione del pre-processing ad ogni audio
2) Estrazione dei coefficienti MFCC
3) Estrazione dei Delta
4) Creazione del file .csv



In [ ]:
header = ["id","path",
          "mfcc1","mfcc2","mfcc3","mfcc4","mfcc5","mfcc6","mfcc7","mfcc8","mfcc9","mfcc10","mfcc11","mfcc12","mfcc13",
          "delta1","delta2","delta3","delta4","delta5","delta6","delta7","delta8","delta9","delta10","delta11","delta12","delta13"]

counter_speaker = 0
final_matrix = []

for dir in os.listdir(BASE_TRAINING_PATH):
    if counter_speaker < 300:
        counter_speaker+=1
        count_audio = 0
        for dir1 in os.listdir(BASE_TRAINING_PATH+dir):
            if count_audio < 40:
                for file in os.listdir(BASE_TRAINING_PATH+dir+"/"+dir1):
                  if count_audio < 40:
                      if not os.path.isdir(BASE_TRAINING_PATH+dir+"/"+dir1+"/"+file):
                        count_audio +=1
                        COMPLETE_PATH = BASE_TRAINING_PATH+dir+"/"+dir1+"/"+file
                        processed_audio, sr = pre_processing(COMPLETE_PATH)  ## 1) applicazione pre-processing

                        mfcc = lb.feature.mfcc(y = processed_audio, sr = sr, n_mfcc=13)  ## 2) estrazione mfcc
                        mfcc_delta = lb.feature.delta(mfcc) ## 3) estrazione delta

                        row_csv = []
                        row_csv.append(dir)
                        row_csv.append(COMPLETE_PATH)


                        for row in mfcc:
                          row_csv_2 = []
                          for value in row:
                            row_csv_2.append(value)
                          row_csv.append(row_csv_2)

                        for row in mfcc_delta:
                          row_csv_2 = []
                          for value in row:
                            row_csv_2.append(value)
                          row_csv.append(row_csv_2)


                        final_matrix.append(row_csv)   ## 4) Creazione csv
                        print("audio n°", count_audio, "done")
    else:
        continue

df = pd.DataFrame(np.array(final_matrix))
df.columns = header
df.to_csv(BASE_PATH + "featuring_extraction.csv" )

<b> Creazione della matrice necessaria per la fase di Vector Quantization. </b>

La matrice sará del tipo:

|         | MFCC_1 | MFCC_2 | ... | MFCC_13 |
|---------|--------|--------|-----|---------|
| FRAME1  |        |        |     |         |
| FRAME2  |        |        |     |         |
| FRAME3  |        |        |     |         |
| FRAME.. |        |        |     |         |




La matrice conterrá in successione, tutti i frame di tutti gli audio del dataset, quindi il numero di righe sará:
$
#ROW = \sum^{300}_{i=0} \sum^{40}_{k=0} frame(k, i)
$
con
1) i = speaker corrente
2) k = audio corrente
3) frame(i, k) = numero frame audio k, speaker i



In [ ]:
df = pd.read_csv(BASE_PATH + "featuring_extraction.csv")

features_mfcc = ["mfcc1", "mfcc2", "mfcc3", "mfcc4", "mfcc5", "mfcc6", "mfcc7", "mfcc8", "mfcc9", "mfcc10", "mfcc11",
            "mfcc12", "mfcc13"]

labels = df.loc[:, ["id"]].values #labael dataset
features_mfcc_values = df.loc[:, features_mfcc].values #mfcc dataset

total = features_mfcc_values.shape[0] * features_mfcc_values.shape[1]
counter = 0
correct_dataset_mfcc = []
header = []
final_matrix = []
frames = []

for i in range(features_mfcc_values.shape[0]): #row_dataset.append(labels[i])
    correct_dataset_mfcc = []
    for j in range(features_mfcc_values.shape[1]):
        row_dataset = []
        features_mfcc_values[i][j] = features_mfcc_values[i][j].split()
        for k in range(len(features_mfcc_values[i][j])):
            features_mfcc_values[i][j][k] = features_mfcc_values[i][j][k].replace("[", "")
            features_mfcc_values[i][j][k] = features_mfcc_values[i][j][k].replace("]", "")
            features_mfcc_values[i][j][k] = features_mfcc_values[i][j][k].replace(",", "")
        for ele in features_mfcc_values[i][j]:
            if ele == "":
                features_mfcc_values[i][j].remove(ele)
        for k in range(len(features_mfcc_values[i][j])):
            features_mfcc_values[i][j][k] = float(features_mfcc_values[i][j][k])
            row_dataset.append(features_mfcc_values[i][j][k])
        correct_dataset_mfcc.append(row_dataset)
    counter += 1

    transposed = np.array(correct_dataset_mfcc).transpose()
    frames.append(transposed.shape[0])
    for row in transposed:
        final_matrix.append(row)

    print(counter, "/", features_mfcc_values.shape[0])


pd.DataFrame(final_matrix).to_csv(BASE_PATH + "preparing_csv_mfcc.csv", header=None, index=None)
pd.DataFrame(frames).to_csv(BASE_PATH + "frame_size_mfcc.csv", header=None, index=None)

<b> Ripetuto processo precedente anche per coefficienti delta </b>

In [ ]:
## Creating delta matrix
df = pd.read_csv(BASE_PATH + "featuring_extraction.csv")

features_delta = ["delta1", "delta2", "delta3", "delta4", "delta5", "delta6", "delta7", "delta8", "delta9", "delta10", "delta11",
            "delta12", "delta13"]

labels = df.loc[:, ["id"]].values #labael dataset
features_delta_values = df.loc[:, features_delta].values #mfcc dataset

total = features_delta_values.shape[0] * features_delta_values.shape[1]
counter = 0
correct_dataset_mfcc = []
header = []
final_matrix = []
frames = []

for i in range(features_delta_values.shape[0]): #row_dataset.append(labels[i])
    correct_dataset_mfcc = []
    for j in range(features_delta_values.shape[1]):
        row_dataset = []
        features_delta_values[i][j] = features_delta_values[i][j].split()
        for k in range(len(features_delta_values[i][j])):
            features_delta_values[i][j][k] = features_delta_values[i][j][k].replace("[", "")
            features_delta_values[i][j][k] = features_delta_values[i][j][k].replace("]", "")
            features_delta_values[i][j][k] = features_delta_values[i][j][k].replace(",", "")
        for ele in features_delta_values[i][j]:
            if ele == "":
                features_delta_values[i][j].remove(ele)
        for k in range(len(features_delta_values[i][j])):
            features_delta_values[i][j][k] = float(features_delta_values[i][j][k])
            row_dataset.append(features_delta_values[i][j][k])
        correct_dataset_mfcc.append(row_dataset)
    counter += 1

    transposed = np.array(correct_dataset_mfcc).transpose()
    frames.append(transposed.shape[0])
    for row in transposed:
        final_matrix.append(row)

    print(counter, "/", features_delta_values.shape[0])


pd.DataFrame(final_matrix).to_csv(BASE_PATH + "preparing_csv_delta.csv", header=None, index=None)
pd.DataFrame(frames).to_csv(BASE_PATH + "frame_size_delta.csv", header=None, index=None)

<b> Fase di Vector Quantization </b>

Operata tramite l'algoritmo K-Means sia per la matrice MFCC che la matrice Delta.

In [ ]:
from sklearn.cluster import KMeans

mfcc = np.array(pd.read_csv(BASE_PATH + "preparing_csv_mfcc.csv", header=None))

model = KMeans(n_clusters=n_cluster)
model.fit(mfcc)

print(model.labels_)
print(len(model.labels_))

labels = model.labels_

pd.DataFrame(labels).to_csv(BASE_PATH + "k" + str(n_cluster)+ "\\mfcc\\labels_quantization.csv", header=None, index=None)

In [ ]:
from sklearn.cluster import KMeans

mfcc = np.array(pd.read_csv(BASE_PATH + "preparing_csv_delta.csv", header=None))

model = KMeans(n_clusters=n_cluster)
model.fit(mfcc)

print(model.labels_)
print(len(model.labels_))

labels = model.labels_

pd.DataFrame(labels).to_csv(BASE_PATH + "k" + str(n_cluster)+ "\\delta\\labels_quantization_delta.csv", header=None, index=None)

<b> Creazione della matrice che conterrá i frame mfcc, rappruppati secondo la fase di Cluster precedente </b>

Ogni riga conterrá l'audio di riferimento, ogni colonna il frame di riferimento per quell'audio, ogni cella conterrá il raggruppamento effettuato dal k-means per il relativo frame del relativo audio.

|           | FRAME 1 | FRAME 2 | FRAME 3 | ... | FRAME_N |
|-----------|---------|---------|---------|-----|---------|
| AUDIO1    | 0       | 5       | 4       |     | 564     |
| AUDIO2    | 24      | 2       | 124     |     | 998     |
| AUDIO...  |         |         |         |     |         |
| AUDIO1200 | 34      | 564     | 121     |     | 123     |

In [ ]:
frames = np.array(pd.read_csv(BASE_PATH + "frame_size_mfcc.csv", header= None)).reshape((1, -1))
labels = np.array(pd.read_csv(BASE_PATH + "k" + str(n_cluster)+ "\\mfcc\\labels_quantization_mfcc.csv", header=None)).reshape((1, -1))

frames = np.array(frames[0])
labels = np.array(labels[0])
vector_complete = []

prec = 0

for index in frames:
    vector_complete.append(labels[prec:prec+index])
    prec += index

vector_complete = np.array(pd.DataFrame(vector_complete))

index = 0
for row in vector_complete:
    row_counter = 0
    for i in range(len(row)):
        if np.isnan(row[i]):
            row[i] = row[row_counter]
            row_counter+=1
    index += 1
    #print(index)
print(vector_complete)

pd.DataFrame(vector_complete).to_csv(BASE_PATH + "k" + str(n_cluster)+ "\\mfcc\\matrix_quantization_filled.csv")


<b> Divisione della matrice mfcc in train-set e test-set </b>

Suddivisione audio <b> per ogni speaker </b>:
1) Train-set = 35 file audio
2) Test-set = 5 file audio



In [ ]:
observations = np.array(pd.read_csv(BASE_PATH + "k" + str(n_cluster)+ "\\mfcc\\matrix_quantization_filled.csv"))
observations = np.delete(observations, 0, axis=1)

index = 0
train_set = []
test_set = []
for row in observations:
    if 0 <= index <= 34:
        train_set.append(row)
        index+=1
    elif index <= 38:
        test_set.append(row)
        index+=1
    elif index == 39:
        test_set.append(row)
        index = 0

test_set = np.array(test_set)
train_set = np.array(train_set)


pd.DataFrame(train_set).to_csv(BASE_PATH + "k" + str(n_cluster)+ "\\mfcc\\train_set_hmm.csv")
pd.DataFrame(test_set).to_csv(BASE_PATH + "k" + str(n_cluster)+ "\\mfcc\\test_set_hmm.csv")

<b> Creazione della matrice che conterrá i frame delta, rappruppati secondo la fase di Cluster precedente </b>

<i> Fase effettuata in maniera identica alla precedente, ma per i coefficienti delta, e non gli mfcc </i>

Ogni riga conterrá l'audio di riferimento, ogni colonna il frame di riferimento per quell'audio, ogni cella conterrá il raggruppamento effettuato dal k-means per il relativo frame del relativo audio.

|           | FRAME 1 | FRAME 2 | FRAME 3 | ... | FRAME_N |
|-----------|---------|---------|---------|-----|---------|
| AUDIO1    | 0       | 5       | 4       |     | 564     |
| AUDIO2    | 24      | 2       | 124     |     | 998     |
| AUDIO...  |         |         |         |     |         |
| AUDIO1200 | 34      | 564     | 121     |     | 123     |

In [ ]:
frames = np.array(pd.read_csv(BASE_PATH + "frame_size_delta.csv", header= None)).reshape((1, -1))
labels = np.array(pd.read_csv(BASE_PATH + "k" + str(n_cluster)+ "\\delta\\labels_quantization_delta.csv", header=None)).reshape((1, -1))

frames = np.array(frames[0])
labels = np.array(labels[0])
vector_complete = []

prec = 0

for index in frames:
    vector_complete.append(labels[prec:prec+index])
    prec += index

vector_complete = np.array(pd.DataFrame(vector_complete))

index = 0
for row in vector_complete:
    row_counter = 0
    for i in range(len(row)):
        if np.isnan(row[i]):
            row[i] = row[row_counter]
            row_counter+=1
    index += 1
    #print(index)
print(vector_complete)

pd.DataFrame(vector_complete).to_csv(BASE_PATH + "k" + str(n_cluster)+ "\\delta\\matrix_quantization_filled_delta.csv")


<b> Divisione della matrice delta in train-set e test-set </b>

Suddivisione audio <b> per ogni speaker </b>:
1) Train-set = 35 file audio
2) Test-set = 5 file audio


In [ ]:
observations = np.array(pd.read_csv(BASE_PATH + "k" + str(n_cluster)+ "\\delta\\matrix_quantization_filled_delta.csv"))
observations = np.delete(observations, 0, axis=1)

index = 0
train_set = []
test_set = []
for row in observations:
    if 0 <= index <= 34:
        train_set.append(row)
        index+=1
    elif index <= 38:
        test_set.append(row)
        index+=1
    elif index == 39:
        test_set.append(row)
        index = 0

test_set = np.array(test_set)
train_set = np.array(train_set)


pd.DataFrame(train_set).to_csv(BASE_PATH + "k" + str(n_cluster)+ "\\delta\\train_set_hmm_delta.csv")
pd.DataFrame(test_set).to_csv(BASE_PATH + "k" + str(n_cluster)+ "\\delta\\test_set_hmm_delta.csv")

<b> Creazione delle matrici di transizione per ognuno dei 300 Modelli di Markov </b>

La creazione di ognuna delle 300 matrici, avviene seguendo la sequenza temporale creata dal k-means. Ogni raggruppamento di frame, operato dal k-means, rappresenterá uno stato della matrice di transizione e l'intera sequenza rappresenta una sequenza di stati.
Ciascun modello apprenderá le probabilitá di transizione, analizzando i 35 audio destinati al relativo speaker nel trainset.

Inoltre verrá prodotta anche un array di probabilitá iniziali, che stimerá per ciascuno speaker la probabilitá che una determinata sequenza inizi con un determinato stato.

In [ ]:
quantization_matrix = pd.read_csv(BASE_PATH +"k" + str(n_cluster)+ "\\mfcc\\train_set_hmm_mfcc.csv")
quantization_matrix = np.array(quantization_matrix, dtype=int)
quantization_matrix = np.delete(quantization_matrix, 0, 1)

## building transition matrix
for person in range(300):
    starting_matrix = np.zeros((n_cluster,))
    observation_matrix = np.zeros((n_cluster, n_cluster))

    for audio_person in range(35):

        actual_audio = quantization_matrix[person*35 + audio_person]
        for frame in range(2806-1):

            if frame == 0:
                starting_matrix[actual_audio[frame]] +=1/35

            observation_matrix[actual_audio[frame]][actual_audio[frame+1]] += 1

    for row in observation_matrix:
        sum_row = sum(row)
        for col in range(len(row)):
            row[col] = row[col] / sum_row


    observation_matrix = pd.DataFrame(observation_matrix).fillna(0)
    observation_matrix.to_csv(BASE_PATH + "k" + str(n_cluster)+ "\\mfcc\\matrix\\quan_mat_" + str(person) +".csv", header=None, index=None)
    starting_matrix = pd.DataFrame(starting_matrix).fillna(0)
    starting_matrix.to_csv(BASE_PATH + "k" + str(n_cluster)+ "\\mfcc\\matrix\\start_mat_" + str(person) +".csv", header=None, index=None)

<b> Creazione delle matrici di transizione per ognuno dei 300 Modelli di Markov </b>

Lo stesso processo avviene per la matrice di coefficienti delta.

In [ ]:
quantization_matrix = pd.read_csv(BASE_PATH +"k" + str(n_cluster)+ "\\delta\\train_set_hmm_delta.csv")
quantization_matrix = np.array(quantization_matrix, dtype=int)
quantization_matrix = np.delete(quantization_matrix, 0, 1)

## building transition matrix
for person in range(300):
    starting_matrix = np.zeros((n_cluster,))
    observation_matrix = np.zeros((n_cluster, n_cluster))

    for audio_person in range(35):

        actual_audio = quantization_matrix[person*35 + audio_person]
        for frame in range(2806-1):

            if frame == 0:
                starting_matrix[actual_audio[frame]] +=1/35

            observation_matrix[actual_audio[frame]][actual_audio[frame+1]] += 1

    for row in observation_matrix:
        sum_row = sum(row)
        for col in range(len(row)):
            row[col] = row[col] / sum_row


    observation_matrix = pd.DataFrame(observation_matrix).fillna(0)
    observation_matrix.to_csv(BASE_PATH + "k" + str(n_cluster)+ "\\delta\\matrix\\quan_mat_" + str(person) +".csv", header=None, index=None)
    starting_matrix = pd.DataFrame(starting_matrix).fillna(0)
    starting_matrix.to_csv(BASE_PATH + "k" + str(n_cluster)+ "\\delta\\matrix\\start_mat_" + str(person) +".csv", header=None, index=None)

<b> Definizione della funzione di calcolo delle probabilitá </b>
<i>Input</i>:
1) observation matrix dello speaker associato
2) starting probability matrix dello speaker associato
3) sequenza di cui si cerca la somiglianza col modello
4) frame attuale, necessario per il calcolo della probabilitá di partenza

<i>Output</i>:
Probabilitá logaritmica che la sequenza assomigli al modello dato in input.

<i>Precisazioni</i>:
Se la sequenza dovesse iniziare o dovesse muoversi in uno stato mai visto dal modello di Markov, la probabilitá del relativo movimento sará $ P(M) = \frac{1}{\text{#stati}} $

In [ ]:
def find_probability(observation_matrix, starting_matrix, sequence, ac_frame):
    prob = 1
    for i in range(len(sequence)-1):
        if ac_frame == 0:
            starting_prob = starting_matrix[sequence[0]]
            if starting_prob == 0:
                starting_prob = 1/len(sequence)
            prob = prob * starting_prob

        mov_prov = observation_matrix[sequence[i]][sequence[i+1]]
        if mov_prov == 0:
            mov_prov = 1/observation_matrix.shape[0]
        prob = prob * mov_prov

    return np.log(prob)

<b> Fase di testing dei modelli </b>

La fase di testing avviene piú volte con un numero di speaker in analisi che varia tra $[2, 3, 5, 10, 50]$


Inoltre, la fase di testing avviene provando successivamente tutte le combinazioni (in sequenza) del numero di speaker scelto in analisi.
Es.
-2 speaker => Coppie di speaker = (1, 2) , (3, 4) , (5, 6) ... etc.
-3 speaker => Terne di speaker = (1, 2, 3) , (4, 5, 6) , (7, 8, 9) ... etc.



1) Per ogni sequenza, corrispondente ad un audio del test-set, verrá calcolato lo score di somiglianza relativo a tutti i modelli in analisi.
2) Ogni sequenza viene divisa in piú finestre contenenti ognuna 20 frame del singolo audio.
3) Su ogni finestra viene computato lo score di somiglianza con tutti i modelli in analisi, lo score piú alto verrá scelto come prediction sulla singola finestra.
4) La fase di calcolo delle prediction, viene effettuata per ogni finestra di ogni sequenza relativa alla martrice mfcc, sia per ogni sequenza relativa alla matrice delta.
5) Tutte le prediction (mfcc, delta) vengono raggruppate e viene scelta come predction finale quella con maggior occorrenza (moda)
6) Viene poi successivamente calcolata la media delle accuracy per ogni coppia/terna etc di speaker in analisi.


In [ ]:

value_nspeaker = []
for person in [2, 3, 5, 10, 50]: ## numero speaker in analisi
    mean = []

    for shift in range(0, 300-person, person): ## scelta della t-pla di speaker
        correct_label = []
        for i in range(person):
            for j in range(5):
                correct_label.append(i)

        matrix = []
        starting_matrix = []

        matrix_delta = []
        starting_matrix_delta = []

        actual = 0
        for i in range(shift, shift+person): ## caricate le observation matrix mfcc e le starting matrix mfcc relative agli speaker in esame
            matrix.append(np.array(pd.read_csv(BASE_PATH + "k" + str(n_cluster)+ "\\mfcc\\matrix\\quan_mat_"+str(i) + ".csv", header=None), dtype=float))
            starting_matrix.append(np.array(pd.read_csv(BASE_PATH + "k" + str(n_cluster)+ "\\mfcc\\matrix\\start_mat_"+str(+i)+".csv", header=None), dtype=float))

        test_set_mfcc = np.array(pd.read_csv(BASE_PATH +"k" + str(n_cluster)+ "\\mfcc\\test_set_hmm_mfcc.csv"), dtype=int) ## caricato il test-set mfcc
        test_set_mfcc = np.delete(test_set_mfcc, 0, axis=1)

        for i in range(shift, shift+person): ## caricate le observation matrix delta e le starting matrix delta relative agli speaker in esame
            matrix_delta.append(np.array(pd.read_csv(BASE_PATH + "k" + str(n_cluster)+ "\\delta\\matrix\\quan_mat_"+str(i) + ".csv", header=None), dtype=float))
            starting_matrix_delta.append(np.array(pd.read_csv(BASE_PATH + "k" + str(n_cluster)+ "\\delta\\matrix\\start_mat_"+str(+i)+".csv", header=None), dtype=float))

        test_set_delta = np.array(pd.read_csv(BASE_PATH +"k" + str(n_cluster)+ "\\delta\\test_set_hmm_delta.csv"), dtype=int) ## caricato il test-set delta
        test_set_delta = np.delete(test_set_delta, 0, axis=1)


        corrected = 0
        test_set_frames_mfcc = test_set_mfcc[shift*5: (shift*5)+5*person]
        test_set_frames_delta = test_set_delta[shift*5: (shift*5)+5*person]

        for audio in range(5*person): ## testato tutti gli audio del testset, relativo al numero di skeaper in esame
            sum_prediction = []
            current = 0
            for frame in range(1,130): ## divisione in finestre
                scores_mfcc = []
                scores_delta = []

                for n_model in range(person):
                    scores_mfcc.append(find_probability(matrix[n_model], starting_matrix[n_model],test_set_frames_mfcc[audio][current:frame*20], current)) ## calcolo log probability sulla finestra n-esima della sequenza mfcc

                for n_model in range(person):
                    scores_delta.append(find_probability(matrix_delta[n_model], starting_matrix_delta[n_model], test_set_frames_delta[audio][current:frame*20], current)) ## calcolo log probability sulla finestra n-esima della sequenza delta

                predict_mfcc = max(scores_mfcc) ## scelta la max probability tra tutte le prediction
                predict_delta = max(scores_delta)

                index_best_mfcc = scores_mfcc.index(predict_mfcc) ## calcolata la prediction relativa alla probabilitá
                index_best_delta = scores_delta.index(predict_delta)

                sum_prediction.append(index_best_mfcc)
                sum_prediction.append(index_best_delta)

                current+=20

            occurrence = Counter(sum_prediction) ## scelta la moda delle prediction
            if correct_label[audio] == occurrence.most_common(1)[0][0]: corrected+=1 
        mean.append(corrected/(5*person))
        #print(corrected/(5*person))


    print("n_person = ", person," sequence = ", mean," max = ", max(mean)," mean = ", np.mean(mean))
    value_nspeaker.append(np.mean(mean))

print(value_nspeaker)